# Setup

## Installs

In [1]:
# Segmentation
# ! pip install --upgrade mxnet
# ! pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
# ! pip install --upgrade gluoncv

# Edge detection
# ! conda install scikit-image

# Depth estimation
# ! pip install -q transformers

## Imports

In [2]:
import numpy as np
import cv2
from os import listdir
from matplotlib import pyplot as plt

In [4]:
# Import custom methods
import general_methods
import semantic_segmentation
import edge_detection
import depth_estimation
import geometry
import transforms

ImportError: attempted relative import with no known parent package

In [ ]:
# Get pre-trained models
seg_model = semantic_segmentation.get_pretrained_model()

In [ ]:
def pipeline(filename, wallpaper_filename):
    input_pil = general_methods.import_and_resize(filename)
    input_img = general_methods.import_mx_image(filename)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    size = (width, height)

    # Segmentation
    mmask = semantic_segmentation.get_segementation(input_img, seg_model)
    labels = general_methods.get_labels_string(mmask, mmask.shape[0])
    walls = general_methods.find_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    other = general_methods.find_not_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    segmented_input = semantic_segmentation.remove_inds(input_img, [other], [0, 0, 0])

    ceiling_x, _ = np.where(labels == "0.47058824,0.47058824,0.3137255,1.0")

    if ceiling_x.size > (0.05 * (height * width)):
        non_ceil_x, non_ceil_y = np.where((labels != "0.47058824,0.47058824,0.47058824,1.0") & (labels != "0.47058824,0.47058824,0.3137255,1.0"))
        non_ceil = np.array([non_ceil_x, non_ceil_y])

        input_copy = np.ones((input_cv2.shape[0], input_cv2.shape[1], 3))
        input_copy = input_copy.astype(float)
        input_copy[non_ceil[0], non_ceil[1]] = np.array([255, 255, 255], dtype=float)
        input_copy[walls[0], walls[1]] = np.array([255, 255, 255], dtype=float)
        input_copy = np.clip(input_copy, 0, 255)

        segmented_input_2 = input_copy.astype(np.uint8)

        for i in range(input_cv2.shape[1]):
            for j in range(input_cv2.shape[0]-1, -1, -1):
                if np.array_equal(segmented_input_2[j][i], [1, 1, 1]):
                    segmented_input_2[:j, i] = np.array([1, 1, 1], dtype=float)
                    break

        edge_map = edge_detection.detect_edges(segmented_input_2)
        edge_map = np.asarray(edge_map.convert("RGB"))

        hough_img = edge_detection.hough_transform(edge_map, input_cv2.shape[0], 10, 0.001, 0.5)
        corners = depth_estimation.get_harris_corners(hough_img)
        labels = general_methods.get_labels_string(corners, 40)
        red = general_methods.find_colour_indices(labels, "255.0,0.0,0.0")
        
        temp2 = corners.copy()
        temp2[red[0], red[1]] = np.array([0, 0, 0], dtype=float)

        ceiling_colours = general_methods.get_labels_string(temp2, temp2.shape[0])
        _, ceiling_corners = np.where(ceiling_colours == "255.0,0.0,0.0")
        corner_inds = geometry.find_corners(ceiling_corners, ceiling_corners, width)
    else:
        # Depth estimation
        depth_image = depth_estimation.estimate_depth(input_pil)

        # Edge detection
        edge_map = edge_detection.detect_edges(input_cv2)
        segmented_edges = edge_detection.get_segmented_edges(edge_map, walls)

        # Find corners
        hough_img = edge_detection.hough_transform(segmented_edges, height, 50, 0.03, 0.02)
        vertical_lines = edge_detection.get_vertical_lines(hough_img)
        hough_colours = general_methods.get_labels_string(vertical_lines, vertical_lines.shape[0])
        hough_corners = edge_detection.get_hough_corners(hough_colours)
        mean_depth = depth_estimation.get_mean_depths(depth_image, other)
        matrix = general_methods.get_matrix(mean_depth)
        matrix_corners = depth_estimation.get_harris_corners(matrix)
        harris_colours = general_methods.get_labels_string(matrix_corners, matrix_corners.shape[0])
        _, harris_corners = np.where(harris_colours == "255,0,0")
        corner_inds = geometry.find_corners(hough_corners, harris_corners, width)
    
    only_walls = geometry.create_wall_corner_map(segmented_input, other, walls, corner_inds)

    # Find room geometry
    contours = geometry.find_contours(only_walls)
    corner_adj_geom = geometry.find_walls(contours, corner_inds)
    new_geom = geometry.find_quadrilaterals(corner_adj_geom, width)
    new_corner_geom = geometry.move_edges_to_corners(new_geom, corner_inds)

    # Perspective transform
    wallpaper = general_methods.import_cv2_image(wallpaper_filename)
    result_1, result_2 = transforms.get_transformed_wallpaper(new_corner_geom, height, width, size, wallpaper)

    # Create final image
    final_mask = transforms.get_wall_mask(new_corner_geom, height, width, walls)
    final_output_1, final_output_2 = transforms.combine_wallpaper_and_input(input_cv2, final_mask, result_1, result_2, walls)
 
    return final_output_1, final_output_2
  

In [ ]:
# # Loop over all images in input folder
# folder_dir = "images/inputs/rooms/small-test-set"
# wallpaper_filename = "images/inputs/wallpaper/check-even.jpg"
 
# for image in listdir(folder_dir):
#     if image[0] >= 'a':
#         filename = "images/inputs/rooms/small-test-set/" + image
#         final_output_1, final_output_2 = pipeline(filename, wallpaper_filename)
#         cv2.imwrite("images/outputs/final-outputs/check-for-breaking/" + image, final_output_1)
#         # cv2.imwrite("images/outputs/final-outputs/test-set/simple-" + image, final_output_2)

In [ ]:
filename = "images/inputs/rooms/test-set/bedroom-4.jpg"
wallpaper_filename = "images/inputs/wallpaper/check-even.jpg"
final_output_1, final_output_2 = pipeline(filename, wallpaper_filename)
cv2.imwrite("images/outputs/final-outputs/bedroom-4.jpg", final_output_1)
# cv2.imwrite("images/outputs/final-outputs/simple-empty6.jpg", final_output_2)